In [ ]:
import wandb
from robust_detection.wandb_config import ENTITY
from robust_detection.models.detr import DETR
from robust_detection.data_utils.rcnn_data_utils import Objects_RCNN
from torchmetrics.detection.map import MeanAveragePrecision
import pandas as pd
import pytorch_lightning as pl
import os
import torch
import numpy as np

In [ ]:
#sweep id: s2oaikka
#auqg6d50
gpu = 0
api = wandb.Api()

results = {}

sweep_dict = {"08k4u8gp":DETR}
#sweep_dict = {("436j7xuk","f9dy36lb"):DETR}
model_names = ["DETR (Hungarian track)"]


#data_dict = {"MMSynthetic":SyntheticMMDataModule, "Pendulum":PendulumDataModule, "CV":CVDataModule}
#data_dict = {"molecules/mol_labels/":Objects_RCNN}#, "mnist/alldigits_2":MNISTCountDataModule,  "mnist/alldigits_5":MNISTCountDataModule} #, \
            #"mnist/alldigits_large":MNISTCountDataModule, "mnist/alldigits_2_large":MNISTCountDataModule,  "mnist/alldigits_5_large":MNISTCountDataModule,}
#data_dict = {"mnist/alldigits_5/":Objects_RCNN}#, "mnist/alldigits_2":MNISTCountDataModule,  "mnist/alldigits_5":MNISTCountDataModule} #, \
            #"mnist/alldigits_large":MNISTCountDataModule
data_dict = {"molecules/molecules_skip":Objects_RCNN}

fold_name = "fold"

In [ ]:
df = pd.DataFrame()

In [ ]:
for i_mod, sweep_name in enumerate(sweep_dict.keys()):
#for i_mod, sweep_names in enumerate(sweep_dict.keys()):
    #print(sweep_name)
    pd_dict_acc = {"Model":model_names[i_mod] + " (Acc)"}
    pd_dict_map = {"Model":model_names[i_mod] + " (mAP)"}


    #model_cls = sweep_dict[sweep_names]
    #sweep_runs = []
    #for sweep_name in sweep_names:
    #    sweep_runs += api.sweep(f"{ENTITY}/object_detection/{sweep_name}").runs
    model_cls = sweep_dict[sweep_name]
    sweep = api.sweep(f"{ENTITY}/object_detection/{sweep_name}")

    for ood in [False,True]:

        pd_dict_acc["Type"] = "OOD" if ood else "In-distribution"
        pd_dict_map["Type"] = "OOD" if ood else "In-distribution"

        
        for data_key in data_dict.keys():

            best_runs = []
            for fold in [0,1,2,3,4]:
                #runs_fold = [r for r in sweep_runs if (r.config.get(fold_name)==fold)]
                runs_fold = [r for r in sweep.runs if (r.config.get(fold_name)==fold)]
                runs_fold_sorted = sorted(runs_fold,key = lambda run: run.summary.get("restored_val_acc"), reverse = True)
                best_runs.append(runs_fold_sorted[0])

            accuracies = []
            mAPs = []
            for run in best_runs:
                fname = [f.name for f in run.files() if "ckpt" in f.name][0]
                run.file(fname).download(replace = True, root = ".")
                model = model_cls.load_from_checkpoint(fname)
                os.remove(fname)

                hparams = dict(model.hparams)
                hparams["re_train"] = False
                hparams["skip_data_path"]=None
                hparams["data_path"]= data_key
                dataset = data_dict[data_key](**hparams)
                dataset.prepare_data()
                trainer = pl.Trainer(logger = False, gpus = 1)

                if ood:
                    preds = trainer.predict(model, dataset.test_ood_dataloader())
                else:
                    preds = trainer.predict(model, dataset.test_dataloader())
                
                Y = []
                Y_hat = []
                scores = []
                boxes = []
                idxs = []
                map_metric = MeanAveragePrecision()
                #print(len(preds))
                for pred in preds:
                    Y += pred["targets"]
                    Y_hat += pred["preds"]
                    scores += pred["scores"]
                    boxes  += pred["boxes"]
                    idxs    += pred["idx"]
                    #pred_map = [dict(boxes=pred["boxes"][i],scores=pred["scores"][i],labels=pred["preds"][i]) for i in range(len(pred))]
                    #target_map = [dict(boxes=pred["boxes_true"][i],labels=pred["targets"][i]) for i in range(len(pred))]
                    #map_metric.update(pred_map,target_map)
                    
                    pred_map = [dict(boxes=pred["boxes"][i],scores=pred["scores"][i],labels=pred["preds"][i]) for i in range(len(pred["targets"]))]
                    target_map = [dict(boxes=pred["boxes_true"][i],labels=pred["targets"][i]) for i in range(len(pred["targets"]))]
                    #print("preds")
                    #print(pred_map)
                    #print("targets")
                    #print(target_map)
                    map_metric.update(pred_map,target_map)
                #print(Y_hat)
                #print(len(Y_hat))
                mAP = map_metric.compute()
                labels = []
                new_boxes = []
                for i in range(len(Y_hat)):
                    keep = [scores[i] > 0.65]
                    labels.append(Y_hat[i][keep])
                    new_boxes.append(boxes[i][keep])
                #print(len(labels))
                #print(labels)
        #keep = [keep.append([preds[i]["scores"] > 0.9]) for i in range(len(targets))]
        #import ipdb; ipdb.set_trace()
        #preds = torch.Tensor([preds[i][preds[i]["scores"] > 0.9] for i in range(len(targets))])
        #import ipdb; ipdb.set_trace()
        #return torch.Tensor([torch.equal(targets[i]["labels"].sort()[0],preds[i]["labels"].sort()[0]) for i in range(len(targets)) ]).mean()
                keep_correct = [torch.equal(Y[i].sort()[0],labels[i][torch.where(labels[i] != 0)].sort()[0]) for i in range(len(Y)) ]
                accuracy = np.array([torch.equal(Y[i].sort()[0],labels[i][torch.where(labels[i] != 0)].sort()[0]) for i in range(len(Y)) ]).mean()
                #accuracy = np.array([torch.equal(Y[i].sort()[0],Y_hat[i].sort()[0]) for i in range(len(Y))]).mean()
                print(accuracy)
                #print(keep_correct)
                print(mAP)
                count_sulfur=0
                id_c = 0
                for idx,correct in zip(idxs, keep_correct):
        
                    if correct:
            
                       boxs    = new_boxes[id_c]
                       b_labels= labels[id_c]
                       if 5 in b_labels:
                            count_sulfur+=1
                    id_c+=1
                print(f"count sulfur in correct image: {count_sulfur}")
                accuracies.append(accuracy)
                mAPs.append(mAP["map"])

            accuracies = np.array(accuracies)
            acc_mu = accuracies.mean()
            acc_std = accuracies.std()
            
            mAPs = np.array(mAPs)
            map_mu = mAPs.mean()
            map_std = mAPs.std()

            acc_str = "$ " + str(acc_mu.round(3))+ "\pm" +str(acc_std.round(3)) +" $"
            map_str = "$ " + str(map_mu.round(3))+ "\pm" +str(map_std.round(3)) +" $"


            pd_dict_acc[data_key] = acc_str
            pd_dict_acc[data_key + " mAP"] = map_str

            print(pd_dict_acc)

        df = df.append(pd_dict_acc,ignore_index =True)

In [ ]:
print(df.loc[df.Model.str.contains("Acc")].to_latex(escape = False,index= False))

In [ ]:
from robust_detection.utils import DATA_DIR
import os
import cv2
from matplotlib import pyplot as plt
count = 0
print(len(idxs))
print(len(keep_correct))
id_c = 0
for idx,correct in zip(idxs, keep_correct):
        
        if correct:
            if count == 37:
                imagename=f"{os.path.join(DATA_DIR,dataset.data_path,'test')}/images/{str(idx)}.png"
                boxs    = new_boxes[id_c]
                b_labels= labels[id_c]
            if 5 in b_labels:
                print(f"silfur in {count}")
            count+=1
        id_c+=1
print(imagename)
print(count)
imgcv = cv2.imread(imagename)
font = cv2.FONT_HERSHEY_SIMPLEX
label_dict={0: "cube_metal_small",1: "cube_metal_large",2: "cube_rubber_small",3: "cube_rubber_large",4: "sphere_metal_small",5: "sphere_metal_large",6: "sphere_rubber_small",7: "sphere_rubber_large",8: "cylinder_metal_small",9: "cylinder_metal_large",10: "cylinder_rubber_small",11: "cylinder_rubber_large"}
for box,label in zip(boxs,b_labels):
    box.cpu().numpy()
    print(int(box[0]))
    cv2.rectangle(imgcv, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (255,0,0), 2)
    cv2.putText(imgcv, str([int(label)-1]), (int(box[0]), int(box[1])), font, 1, (0, 255, 0), 1, cv2.LINE_AA)
plt.imshow(imgcv)
plt.show()